In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

<H1> Функция для запроса данных с бесконечно пролистываемой страницы

In [2]:
def get_Comics_List(page: int):
    json ={
        "sort":"rate",
        "dir":"desc",
        "page":page,
        "types":["9", "8"],
        "site_id":"1",
        "type":"manga",
        "caution_list":["Отсутствует", "16+", "18+"]
    } # json для полечения комиксов западных и русских
    headers ={
        "X-Csrf-Token":None,
        "X-Requested-With":"XMLHttpRequest",
        "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/118.0"
    }
    url = "https://mangalib.me/api/list"

    session = requests.session()

    html = session.get("https://mangalib.me") #Получаем базовую страницу
    soup = BeautifulSoup(html.text, 'lxml')
    csrf_token = soup.find("meta", {"name":"_token"}).get("content")
    headers["X-Csrf-Token"] = csrf_token #Запоминаем токен

    session.headers.update(headers)
    manga_list = session.post(url, json=json) #Получаем данные
    return manga_list.json()

<h4> Считываем первые 600 записей

In [3]:
manga_mas = []
for i in range(10):
    manga_mas.append(get_Comics_List(i+1))
manga_mas

[{'items': {'current_page': 1,
   'data': [{'id': 8135,
     'name': 'Love Advice from the Great Duke of Hell',
     'eng_name': '',
     'rus_name': 'Любовный совет от великого герцога из Ада',
     'cover': '9wQk93wUnvzr',
     'slug': 'love-advice-from-the-great-duke-of-hell',
     'type_id': 9,
     'created_at': '2018-12-14T05:32:54.000000Z',
     'updated_at': '2023-12-07T17:23:32.000000Z',
     'site': 1,
     'type': 'Комикс западный',
     'modelType': 'manga',
     'covers': {'default': 'https://cover.imglib.info/uploads/cover/love-advice-from-the-great-duke-of-hell/cover/9wQk93wUnvzr_250x350.jpg',
      'thumbnail': 'https://cover.imglib.info/uploads/cover/love-advice-from-the-great-duke-of-hell/cover/9wQk93wUnvzr_thumb.jpg'},
     'coverImage': 'https://cover.imglib.info/uploads/cover/love-advice-from-the-great-duke-of-hell/cover/9wQk93wUnvzr_250x350.jpg',
     'coverImageThumbnail': 'https://cover.imglib.info/uploads/cover/love-advice-from-the-great-duke-of-hell/cover/9wQk

Смотрим данные, которые мы вытянули из запросов

In [4]:
manga_mas[0]['items']['data'][0].keys()

dict_keys(['id', 'name', 'eng_name', 'rus_name', 'cover', 'slug', 'type_id', 'created_at', 'updated_at', 'site', 'type', 'modelType', 'covers', 'coverImage', 'coverImageThumbnail', 'href', 'title_status_id'])

<h2> Заполняем DF

In [5]:
from random import random
from time import sleep
href_mas = []
data_mas = []
df = pd.DataFrame()
i = 0
for manga_mas_lower in manga_mas:
    data = manga_mas_lower['items']['data']
    
    for elems in data:
        for key in list(elems.keys()):
            # print(elems[key])
            if key != 'covers':
                df.loc[i, key] = elems[key]
            else:
                for kkey in elems[key].keys():
                    df.loc[i, "cover_"+kkey] = elems[key][kkey]
        # df.loc[i,'href'] = elems['href']
        # df.loc[i,'id'] = int(elems['id'])
        i+=1
df.head()


,id,name,eng_name,rus_name,cover,slug,type_id,created_at,updated_at,site,type,modelType,cover_default,cover_thumbnail,coverImage,coverImageThumbnail,href,title_status_id
0,8135.0,Love Advice from the Great Duke of Hell,,Любовный совет от великого герцога из Ада,9wQk93wUnvzr,love-advice-from-the-great-duke-of-hell,9.0,2018-12-14T05:32:54.000000Z,2023-12-07T17:23:32.000000Z,1.0,Комикс западный,manga,https://cover.imglib.info/uploads/cover/love-a...,https://cover.imglib.info/uploads/cover/love-a...,https://cover.imglib.info/uploads/cover/love-a...,https://cover.imglib.info/uploads/cover/love-a...,https://mangalib.me/love-advice-from-the-great...,NaN
1,19009.0,The Little Trashmaid,The Little Trashmaid,Маленькая мусорная русалочка,KvRCkR9vnCfL,the-little-trashmaid,9.0,2019-11-30T09:31:44.000000Z,2023-12-07T17:23:47.000000Z,1.0,Комикс западный,manga,https://cover.imglib.info/uploads/cover/the-li...,https://cover.imglib.info/uploads/cover/the-li...,https://cover.imglib.info/uploads/cover/the-li...,https://cover.imglib.info/uploads/cover/the-li...,https://mangalib.me/the-little-trashmaid,NaN
2,45864.0,Тёмный замок,The Dark Castle,Тёмный Замок,8tyXnxq0GvGc,temnyy-zamok,8.0,2020-08-30T09:02:21.000000Z,2023-12-07T17:22:03.000000Z,1.0,Руманга,manga,https://cover.imglib.info/uploads/cover/temnyy...,https://cover.imglib.info/uploads/cover/temnyy...,https://cover.imglib.info/uploads/cover/temnyy...,https://cover.imglib.info/uploads/cover/temnyy...,https://mangalib.me/temnyy-zamok,NaN
3,9545.0,Brutus &amp; Pixie,Brutus And Pixie,Брут и Пикси,sMeSbOWbTEOi,brutus-and-pixie,9.0,2019-02-11T15:28:42.000000Z,2023-12-07T17:13:15.000000Z,1.0,Комикс западный,manga,https://cover.imglib.info/uploads/cover/brutus...,https://cover.imglib.info/uploads/cover/brutus...,https://cover.imglib.info/uploads/cover/brutus...,https://cover.imglib.info/uploads/cover/brutus...,https://mangalib.me/brutus-and-pixie,NaN
4,20594.0,Confused cat,,Озадаченный кот за столом,cover,ozadachennyy-kot-za-stolom,9.0,2020-01-04T19:53:19.000000Z,2023-12-07T17:11:01.000000Z,1.0,Комикс западный,manga,https://cover.imglib.info/uploads/cover/ozadac...,https://cover.imglib.info/uploads/cover/ozadac...,https://cover.imglib.info/uploads/cover/ozadac...,https://cover.imglib.info/uploads/cover/ozadac...,https://mangalib.me/ozadachennyy-kot-za-stolom,NaN


Функция для красивого статус бара

In [6]:
def log_progress(sequence, every=None, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )


<h2>Читаем данные со страниц комиксов

In [7]:
import sys


aim_heads = {'Год релиза':'Release_Year', 'Статус тайтла':'Title_Status', 'Статус перевода':'Translate_Status', 
             'Автор':'Author', 'Художник':'Artist',  'Возрастной рейтинг':'Age_restriction', 'Загружено глав':'Chapters_Loaded'}
for href in enumerate(log_progress(df['href'], every = 1)):
    heads = []
    bodies = {}
    response = requests.get(href[1])
    bs = BeautifulSoup(response.text)
    heads_bs = bs.find_all('div', {"class":"media-info-list__title"})
    bodies_bs = bs.find_all('div', {"class":"media-info-list__value"})
    
    for i in heads_bs:
        heads.append(i.text)
    for i in enumerate(heads):
        if i[1] in aim_heads.keys() :
            # print(aim_heads[i[1]])
            bodies[aim_heads[i[1]]] = bodies_bs[i[0]].text.replace('\n','') #''.join(filter(str.isalnum, bodies_bs[i[0]].text))
    # print(heads)
    # print(bodies)
    for i in bodies.keys():
        df.loc[href[0], i] = bodies[i]

Описание итогового DF

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 600 entries, 0 to 599
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   600 non-null    float64
 1   name                 600 non-null    object 
 2   eng_name             600 non-null    object 
 3   rus_name             600 non-null    object 
 4   cover                600 non-null    object 
 5   slug                 600 non-null    object 
 6   type_id              600 non-null    float64
 7   created_at           600 non-null    object 
 8   updated_at           600 non-null    object 
 9   site                 600 non-null    float64
 10  type                 600 non-null    object 
 11  modelType            600 non-null    object 
 12  cover_default        600 non-null    object 
 13  cover_thumbnail      600 non-null    object 
 14  coverImage           600 non-null    object 
 15  coverImageThumbnail  600 non-null    obj